# Big Data Analytics: NYC Crashes

### Installation der Python Packages

In [ ]:
pip install kafka-python pymongo

In [ ]:
from kafka import KafkaProducer
from kafka import KafkaConsumer


producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

dataset = open('Motor_Vehicle_Collisions_-_Crashes.csv', encoding='utf-8')
rows = dataset.readlines()[1:]

for i, row in enumerate(rows):
        producer.send('nyc_crashes', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

consumer = KafkaConsumer('nyc_crashes', bootstrap_servers=['localhost:9092'], auto_offset_reset="earliest")


for msg in consumer:
    print(msg.value.decode('utf-8'))

In [ ]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

for i in range(1, 13):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2019-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

for i in range(1, 7):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2020-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

In [ ]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import datetime as dt

consumer = KafkaConsumer('yellow-cabs', bootstrap_servers=[""])

client = MongoClient("")

yellow_collection = client['datawarehouse']['bg-yellowcabs']
yellow_collection.delete_many({})

count = 0

for msg in consumer: 
    count += 1
    print('Received new message: %s' % count)
    values = msg.value.decode('utf-8').split(',')
    
    yellow_collection.insert_one({
        'pickup_datetime': dt.datetime.strptime(values[1], "%Y-%m-%d %H:%M:%S"),
        'dropoff_datetime': dt.datetime.strptime(values[2], "%Y-%m-%d %H:%M:%S"),
        'passenger_count': int(values[3]),
        'trip_distance': float(values[4]),
        'PULocationID': values[5],
        'DOLocationID': values[6],
        'payment_type': int(values[9]),
        'fare_amount': float(values[10]),
        'tip_amount': float(values[15]),
        'total_amount': float(values[16])
    })